In [1]:
from Helper.Load.load import Base_Load
from Helper.Load.supervised import Supervised
from Helper.Load.regression import Regression
from Helper.Load.classification import Classification
from Helper.Visual.Visual import Visual
from Helper.Visual.classification import VClassification
from Helper.Transform.Transform.classification import Classificaton_Transform
from Helper.Train.Train import Train

from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
loadC = Classification("Data", "haberman.dat", "haberman-names.txt")
transformC = Classificaton_Transform(loadC)
trainC = Train(loadC, Classificaton_Transform, RandomForestClassifier)

In [3]:
pipe = trainC.pipeline

In [4]:
pipe.get_params().keys()

dict_keys(['memory', 'steps', 'pipeline', 'estimator', 'pipeline__memory', 'pipeline__steps', 'pipeline__inputs', 'pipeline__inputs__n_jobs', 'pipeline__inputs__transformer_list', 'pipeline__inputs__transformer_weights', 'pipeline__inputs__Age', 'pipeline__inputs__Year', 'pipeline__inputs__Positive', 'pipeline__inputs__Age__memory', 'pipeline__inputs__Age__steps', 'pipeline__inputs__Age__Age_selector', 'pipeline__inputs__Age__KBinsDiscretizer_transform', 'pipeline__inputs__Age__Age_selector__key', 'pipeline__inputs__Age__KBinsDiscretizer_transform__encode', 'pipeline__inputs__Age__KBinsDiscretizer_transform__n_bins', 'pipeline__inputs__Age__KBinsDiscretizer_transform__strategy', 'pipeline__inputs__Year__memory', 'pipeline__inputs__Year__steps', 'pipeline__inputs__Year__Year_selector', 'pipeline__inputs__Year__KBinsDiscretizer_transform', 'pipeline__inputs__Year__Year_selector__key', 'pipeline__inputs__Year__KBinsDiscretizer_transform__encode', 'pipeline__inputs__Year__KBinsDiscretizer_

In [5]:
from sklearn.model_selection import GridSearchCV
hyperparameters = {
                   'estimator__max_depth': [1,2,4,8], 
                   'estimator__max_depth': [1,2,4,8],
                   'estimator__n_estimators':[100,110,120],
                   'estimator__class_weight':['balanced'],
                   'pipeline__inputs__Age__KBinsDiscretizer_transform__n_bins':range(17,20),
                   'pipeline__inputs__Year__KBinsDiscretizer_transform__n_bins':range(17,20),
                   'pipeline__inputs__Positive__KBinsDiscretizer_transform__n_bins':range(17,20)
                  }
hyperparameters_2 = {
                   'estimator__gamma': np.linspace(.25, 2, 20), 
                   'estimator__kernel': ['rbf', 'linear'],
                   'estimator__probability':[True],
                   'estimator__class_weight':['balanced'],
                   'pipeline__inputs__Age__KBinsDiscretizer_transform__n_bins':[5,7,10,14],
                   'pipeline__inputs__Year__KBinsDiscretizer_transform__n_bins':[5,7,10,14],
                   'pipeline__inputs__Positive__KBinsDiscretizer_transform__n_bins':[5,7,10,14]
                  }

In [6]:
clf = trainC.construct_GSCV(hyperparameters)
clf.fit(loadC.partition.X_train, loadC.partition.y_train.values.ravel())

/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


GridSearchCV(cv=10, error_score='raise-deprecating',
       estimator=Pipeline(memory=None,
     steps=[('pipeline', Pipeline(memory=None,
     steps=[('inputs', FeatureUnion(n_jobs=None,
       transformer_list=[('Age', Pipeline(memory=None,
     steps=[('Age_selector', NumberSelector(key='Age')), ('KBinsDiscretizer_transform', KBinsDiscretizer(encode='ordinal', n_bins=15, strategy='uniform'))...obs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False))]),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'estimator__max_depth': [1, 2, 4, 8], 'estimator__n_estimators': [100, 110, 120], 'estimator__class_weight': ['balanced'], 'pipeline__inputs__Age__KBinsDiscretizer_transform__n_bins': range(17, 20), 'pipeline__inputs__Year__KBinsDiscretizer_transform__n_bins': range(17, 20), 'pipeline__inputs__Positive__KBinsDiscretizer_transform__n_bins': range(17, 20)},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',


In [7]:
clf.best_params_

{'estimator__class_weight': 'balanced',
 'estimator__max_depth': 1,
 'estimator__n_estimators': 110,
 'pipeline__inputs__Age__KBinsDiscretizer_transform__n_bins': 19,
 'pipeline__inputs__Positive__KBinsDiscretizer_transform__n_bins': 19,
 'pipeline__inputs__Year__KBinsDiscretizer_transform__n_bins': 19}

In [8]:
clf.refit

True

In [9]:
preds = clf.predict(loadC.partition.X_test)
probs = clf.predict_proba(loadC.partition.X_test)
np.mean(preds == loadC.partition.y_test.values)

0.5898441329281443

In [10]:
from sklearn.metrics import confusion_matrix, classification_report
confusion_matrix(loadC.partition.y_test.values, preds)

array([[57, 17],
       [13, 14]])

In [11]:
print(classification_report(loadC.partition.y_test.values, preds))

              precision    recall  f1-score   support

    negative       0.81      0.77      0.79        74
    positive       0.45      0.52      0.48        27

   micro avg       0.70      0.70      0.70       101
   macro avg       0.63      0.64      0.64       101
weighted avg       0.72      0.70      0.71       101

